### Problema de una barra finita 1D
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VladP008/NotasEDPs/blob/main/EcuacionesParabolicas/Problemas-1D/CondicionMixta/ProblemaBarra1D-Mixta.ipynb)

Se tiene el siguiente problema con valores iniciales y condiciones de frontera Mixta, es decir, de tipo Dirichlet en $x=0$ y de 
tipo Neumann en $x=\pi$
\begin{equation}
   \left\{
      \begin{aligned}
        u_{t} &= ku_{xx} &&0<x<\pi, \quad 0<t, \\
        u(x,0) &=f(x) && 0<x<\pi, \\
        u(0,t) &=0 && 0<t,\\
        u_x(\pi,t) &= 0 && 0<t.
      \end{aligned}
    \right.
\end{equation}

Donde $f$ es continua a trozos

La solución de este problema está dada por 
\begin{equation*}
    \boxed{u(x, t)=\sum_{n=1}^{\infty}B_n e^{(-(\frac{n}{2})^2kt)}\sin\big(\frac{n}{2}x\big)}
\end{equation*}
Donde los coeficientes $B_n$ se obtienen al considerar la serie de senos de $\bar{f}$, que es la extensión de $f$ sobre $(0,2\pi)$ respecto a la recta $x=\pi$. Es decir,
\begin{equation*}
    \tilde{f}(x)= \left\{ \begin{array}{lcc}
                f(x) &   si  & 0<x<\pi, \\
                f(2\pi-x) &  si & \pi<x < 2\pi. \\
                \end{array}
    \right.
\end{equation*}
y
\begin{equation*}
   \tilde{f}(x) =\sum_{n=1}^{\infty}B_n \sin\big(\frac{n}{2}x\big)
\end{equation*}

#### Código para visualizar el fénomeno
- Objetivo: visualizar el comportamiento de la solución en tiempo y apreciar como es el flujo de calor en una barra 1-dimensional

In [6]:
# Se utilizan las siguientes bibliotecas:
import matplotlib.pyplot as plt    # Para las gráficas
import numpy as np                 # Para el uso de arrays
import matplotlib.animation as animation    # Para la animación
from IPython.display import HTML            # Para poder visualizar la animación en el notebook


# La siguiente instrucción se utiliza para habilitar el modo interactivo en el notebook
%matplotlib notebook

# Parámetros del problema
k = 0.01                  # Constante de conductividad térmica
l = np.pi                 # Longitud de la barra (π)
N = 2000                  # Número de puntos para la discretización del dominio espacial
x = np.linspace(0,l, N + 1) # Puntos en el intervalo [0,l]
tmax = 500                      # Tiempo máximo de la animación
t = np.linspace(0, tmax, 50)   # Vector del tiempo
N_s = 20    # Número de términos para la serie 

def a_n(n):
    """Función que regresa el n-ésimo coeficiente de Fourier de la serie
    de en senos de una función en particular.

    Parametros
    ----------
    n: int
        Número natural mayor o igual a 1.
    
    Returns
    -------
    coef: float 
        El n-ésimo coeficiente de Fourier.
    """
    ###########################################################
    # Coeficinte B_n de la función f(x) = x
    coef = (8/np.pi)*((-1)**(n+1))/(2*n-1)**2
    ###########################################################
    return coef

def u(x,t):
    """Función que regresa el valor u(x,t) hasta un número determinado de terminos N_s

    Parametros
    ----------
    x: array
        Puntos en el dominio espacial.
    t: float
        Punto en el dominio temporal
    
    Returns
    -------
    y: array 
        Solución u(x,t)
    """
    y = 0
    for i in range(1,N_s):
        y = y + a_n(i)*np.exp(-(k/4)*t*(2*i-1)**2)*np.sin((2*i-1)*x/2)
    return y
    
# Se crea la figura y se fijan parámetros
fig= plt.figure()
gs = fig.add_gridspec(2, 1, hspace=0.15, wspace=0)
(ax), (ax2) = gs.subplots(sharex='col', sharey='row')
line, = ax.plot(x, u(x, t[0]), color='blue', linewidth = 2) # Primera imagen
text = ax.text(l/2, 4.0, f't = {t[0]}', fontsize=10, color='black', ha='center', va='center',
              bbox=dict(facecolor = 'None',alpha=0.5,edgecolor='blue', boxstyle='round'))  # Para mostrar el tiempo en la animación
mapa_calor = ax2.imshow(u(x,0).reshape(1, N+1), aspect='auto', cmap='plasma', origin='lower', extent=[0, l, 0, 0.5])

# Añadir una barra de color
calor_bar = plt.colorbar(mapa_calor, ax=ax2)
calor_bar.set_label('Temperatura')

# Ajustes de los ejes, etiquetas y título
ax.set_title('Problema de calor en una barra 1-D\nCondición de frontera Mixta')
ax.grid(color='grey', linestyle='-', linewidth=0.1)
ax.set_ylim([-1, 5])      # Se fijan los límites del eje y
ax.set_xlim([0, l]) # Se fijan los límites del eje x
#ax.set_xlabel('x')

#ax.text(0.05, 4.0,  r'$u(x,0) = \frac{x^3}{8}$')
ax.text(0.05, 4.5,  f'k = {k}')
ax2.text(l/2,-0.25,'Barra', horizontalalignment='center')
ax2.set_ylim([-0.25, 0.75])      # Se fijan los límites del eje y
ax2.set_xlim([-0.25, l+0.25]) # Se fijan los límites del eje x
ax2.set_xlabel('x')
ax2.axes.set_axis_off()
# Función de actualización de cada frame
def update(frame):
    """Función que regresa el valor de la solución para cada frame de la animación
    
    Parametros
    ----------
    frame : int
        Indice del arreglo del tiempo

    """
    y = u(x, t[frame])
    line.set_ydata(y)
    text.set_text(f't= {round(t[frame],1)}')

    # Actualizar la imagen con la nueva distribución de temperatura
    mapa_calor.set_array(y.reshape(1, N+1))  # Reshape para mostrarlo como una fila
    

# Crear la animación
animacion = animation.FuncAnimation(fig, update, frames=len(t), interval=200)

# Mostrar la animación en el notebook
HTML(animacion.to_jshtml())

# Se guarda en formato gif
#animacion.save(filename="Barra-1D-DC-NC_ejemplo1.gif", writer="pillow")

<IPython.core.display.Javascript object>